# Data Setup

### Creating The External Table From Data in The Bucket
```sql
CREATE OR REPLACE EXTERNAL TABLE `dezoomcamp-486221.de_zoomcamp.yellow_external_table`
OPTIONS (
  format = 'PARQUET',
  uris = ['gs://taxi-rides-ny-de/*.parquet']
);
```

### Creating The Materialized Table From External Table
```sql
CREATE OR REPLACE TABLE `dezoomcamp-486221.de_zoomcamp.yellow_materialized_table` AS 
SELECT * FROM `dezoomcamp-486221.de_zoomcamp.yellow_external_table`
```

Screenshot:

![data_prep1.png](data_prep1.png)
![data_prep2.png](data_prep2.png)

## Question 1. Counting records

What is count of records for the 2024 Yellow Taxi Data?
- 65,623
- 840,402
- 20,332,093
- 85,431,289

## Answer: - 20,332,093

```sql
SELECT count (*)
FROM `dezoomcamp-486221.de_zoomcamp.yellow_external_table`
```

Screenshot:

![q1.png](q1.png)

## Question 2. Data read estimation

Write a query to count the distinct number of PULocationIDs for the entire dataset on both the tables.
 
What is the **estimated amount** of data that will be read when this query is executed on the External Table and the Table?

- 18.82 MB for the External Table and 47.60 MB for the Materialized Table
- 0 MB for the External Table and 155.12 MB for the Materialized Table
- 2.14 GB for the External Table and 0MB for the Materialized Table
- 0 MB for the External Table and 0MB for the Materialized Table

## Answer: - 0 MB for the External Table and 155.12 MB for the Materialized Table

```sql
SELECT count (distinct PULocationID)
FROM `dezoomcamp-486221.de_zoomcamp.yellow_external_table`;
```

Screenshot1:

![q2.1.png](q2.1.png)

```sql
SELECT count (distinct PULocationID)
FROM `dezoomcamp-486221.de_zoomcamp.yellow_materialized_table`;
```

Screenshot2:

![q2.2.png](q2.2.png)

## Question 3. Understanding columnar storage

Write a query to retrieve the PULocationID from the table (not the external table) in BigQuery. Now write a query to retrieve the PULocationID and DOLocationID on the same table.

Why are the estimated number of Bytes different?
- BigQuery is a columnar database, and it only scans the specific columns requested in the query. Querying two columns (PULocationID, DOLocationID) requires 
reading more data than querying one column (PULocationID), leading to a higher estimated number of bytes processed.
- BigQuery duplicates data across multiple storage partitions, so selecting two columns instead of one requires scanning the table twice, 
doubling the estimated bytes processed.
- BigQuery automatically caches the first queried column, so adding a second column increases processing time but does not affect the estimated bytes scanned.
- When selecting multiple columns, BigQuery performs an implicit join operation between them, increasing the estimated bytes processed

## Answer: - BigQuery is a columnar database, and it only scans the specific columns requested in the query. Querying two columns (PULocationID, DOLocationID) requires reading more data than querying one column (PULocationID), leading to a higher estimated number of bytes processed.

```sql
SELECT 
PULocationID
FROM `dezoomcamp-486221.de_zoomcamp.yellow_materialized_table`;
```

Screenshot1:

![q3.1.png](q3.1.png)

```sql
SELECT 
PULocationID,DOLocationID
FROM `dezoomcamp-486221.de_zoomcamp.yellow_materialized_table`;
```

Screenshot2:

![q3.2.png](q3.2.png)

## Question 4. Counting zero fare trips

How many records have a fare_amount of 0?
- 128,210
- 546,578
- 20,188,016
- 8,333

## Answer: - 8,333

```sql
SELECT count(*)
FROM `dezoomcamp-486221.de_zoomcamp.yellow_materialized_table`
WHERE fare_amount=0;
```

Screenshot:

![q4.png](q4.png)

## Question 5. Partitioning and clustering

What is the best strategy to make an optimized table in Big Query if your query will always filter based on tpep_dropoff_datetime and order the results by VendorID (Create a new table with this strategy)

- Partition by tpep_dropoff_datetime and Cluster on VendorID
- Cluster on by tpep_dropoff_datetime and Cluster on VendorID
- Cluster on tpep_dropoff_datetime Partition by VendorID
- Partition by tpep_dropoff_datetime and Partition by VendorID

## Answer: - Partition by tpep_dropoff_datetime and Cluster on VendorID

```sql
CREATE OR REPLACE TABLE `dezoomcamp-486221.de_zoomcamp.yellow_part_clus_table`
PARTITION BY DATE(tpep_dropoff_datetime)
CLUSTER BY VendorID AS
SELECT * 
FROM `dezoomcamp-486221.de_zoomcamp.yellow_materialized_table`;
```

Screenshot:

![q5.png](q5.png)

## Question 6. Partition benefits

Write a query to retrieve the distinct VendorIDs between tpep_dropoff_datetime
2024-03-01 and 2024-03-15 (inclusive)


Use the materialized table you created earlier in your from clause and note the estimated bytes. Now change the table in the from clause to the partitioned table you created for question 5 and note the estimated bytes processed. What are these values? 


Choose the answer which most closely matches.
 

- 12.47 MB for non-partitioned table and 326.42 MB for the partitioned table
- 310.24 MB for non-partitioned table and 26.84 MB for the partitioned table
- 5.87 MB for non-partitioned table and 0 MB for the partitioned table
- 310.31 MB for non-partitioned table and 285.64 MB for the partitioned table

## Answer: - 310.24 MB for non-partitioned table and 26.84 MB for the partitioned table

```sql
SELECT distinct VendorID
FROM `dezoomcamp-486221.de_zoomcamp.yellow_materialized_table`
WHERE tpep_dropoff_datetime >='2024-03-01 00:00:00'
and tpep_dropoff_datetime <='2024-03-16 00:00:00';
```

Screenshot1:

![q6.1.png](q6.1.png)

```sql
SELECT distinct VendorID
FROM `dezoomcamp-486221.de_zoomcamp.yellow_part_clus_table`
WHERE tpep_dropoff_datetime >='2024-03-01 00:00:00'
and tpep_dropoff_datetime <='2024-03-16 00:00:00';
```

Screenshot2:

![q6.2.png](q6.2.png)

## Question 7. External table storage

Where is the data stored in the External Table you created?

- Big Query
- Container Registry
- GCP Bucket
- Big Table

## Answer: - GCP Bucket

https://docs.cloud.google.com/bigquery/docs/external-data-cloud-storage

## Question 8. Clustering best practices

It is best practice in Big Query to always cluster your data:
- True
- False

## Answer: False

This would depend on the size of the data, as Tables that are less than 1 GB don’t show significant improvement with partitioning and clustering. Partitioning can greatly help with costs, reducing the amount of data that needs to be read and returned, but clustering is an option that may or may not enhance performance depending on how often users query with filters and aggregates on multiple columns, and it's cost benefit is not known before running the queries.

https://docs.google.com/presentation/d/1a3ZoBAXFk8-EhUsd7rAZd-5p_HpltkzSeujjRGB2TAI/edit?usp=sharing

## Question 9. Understanding table scans

No Points: Write a `SELECT count(*)` query FROM the materialized table you created. How many bytes does it estimate will be read? Why?

## Answer: It estimates that there will be 0B read from the table, because it's an external table, and BigQuery does not have access to the table storage statistics that would provide the size and row count that would normally be available in the internal dataset. 

https://docs.cloud.google.com/bigquery/docs/external-tables

```sql
SELECT count (*)
FROM `dezoomcamp-486221.de_zoomcamp.yellow_materialized_table`
```

Screenshot:

![q9.png](q9.png)